# Pipeline SWV → FOPDT → PD (Explicação Detalhada)

Este notebook aprofunda cada etapa do pipeline implementado pelos scripts do repositório, trazendo fórmulas, decisões de projeto, limitações e boas práticas.

Etapas:
1) Sanitização de logs SWV/ITM → CSV canônico (`filter_swv_export.py`)
2) Cálculo de séries derivadas e métricas temporais (`analyze_step_responses.py`)
3) Identificação FOPDT (K, L, τ) por método temporal (crossings 5% e 63,2%)
4) Síntese PD via Ziegler–Nichols (curva de reação) e escala para firmware
5) Emissão de summary + CSVs derivados
6) Consolidação em `tuning_profiles.py` para simulação/firmware

Incluímos ainda um glossário de unidades, um checklist de diagnóstico e observações sobre a relação CPR do encoder ↔ Kp.

## Glossário e Unidades
- `pulses` (DDA): contagem de passos emitidos (contador crescente).
- `encoder`: contagem de pulsos do encoder (aqui tratada como absoluta na sanitização).
- `time_ms`: tempo em milissegundos.
- `v_cmd` [pulsos/s]: velocidade do comando (derivada de `pulses`).
- `v_enc` [contagens/s]: velocidade medida (derivada de `encoder`).
- `K` [contagens/passo]: ganho estático (escala entre comando e medição).
- `L` [s]: tempo morto (dead-time).
- `τ` [s]: constante de tempo do primeiro ordem.
- `Kp, Ki, Kd`: ganhos contínuos do PD (Ki=0 na malha de velocidade).
- `kp_i, ki_i, kd_i`: ganhos inteiros do firmware, com escala `K_SCALE=256`.

## 1) Sanitização (Regras Detalhadas)
Objetivo: eliminar ruído textual e inconsistências para produzir CSVs confiáveis.

Regras aplicadas por `filter_swv_export.py`:
1. Formato estrito por linha: `axis,id,time_ms,encoder,pulses` — todos inteiros (regex numérica).
2. Eixo único por arquivo: o primeiro `axis` válido define o eixo; divergências são descartadas.
3. Monotonia e progressão:
   - `id`: estritamente crescente (`seq_k > seq_{k-1}`).
   - `time_ms` e `pulses`: não decrescentes (`t_k ≥ t_{k-1}` e `pulses_k ≥ pulses_{k-1}`).
4. Encoder absoluto: substitui `encoder` por `abs(encoder)` para evitar inversões de sinal de logging.
5. Arquivo vazio após filtros → o arquivo de saída é removido.

Racional:
- Backtracks em `time_ms` ou `pulses` corrompem derivadas (divisão por Δt ≤ 0), então descartamos.
- `id` estrito facilita detectar perdas/repetições de linhas.
- Eixo único evita misturar constantes físicas (CPR, microstep).
- `abs(encoder)` estabiliza medições quando o log exporta contagens relativas com sinal ambiguamente definido.

Exemplos (aceita/descarta):
- Aceita: `0,15,12345,400,1000` (numérica, monotônica, eixo consistente).
- Descarta: `0,15,12340,410,1001` (time_ms regressivo).
- Descarta: `1,16,12350,420,1002` (mudança de eixo após fixar `0`).

## 2) Séries Derivadas e Escolhas de Janela
Dadas as linhas `k-N+1` e `k`:

$t_k = time\_ms(k)/1000$,  $\Delta t_k = t_k - t_{k-N+1}$,  $\Delta p_k = pulses(k) - pulses(k-N+1)$,  $\Delta e_k = encoder(k) - encoder(k-N+1)$.

$v_{cmd}(k) = \tfrac{\Delta p_k}{\Delta t_k}$,   $v_{enc}(k) = \tfrac{\Delta e_k}{\Delta t_k}$.

- Série bruta (N=2): sensível a ruído, rápido para detectar o degrau.
- Janela deslizante (N>2): reduz ruído; `N=10` é um bom compromisso para SWV típico.

Trade-offs ao escolher `N` (parâmetro `--stride`):
- `N` pequeno: mais responsivo, porém mais ruidoso.
- `N` grande: mais suave, porém pode atrasar e achatar o degrau.

Dicas:
- Se `Δt` mínimo for alto (telemetria lenta), use `N` menor.
- Se o encoder apresentar jitter, aumente `N`.
- Sempre valide a forma com plots quando ajustar `N`.

## 3) Identificação FOPDT (K, L, τ)
Modelo: $G_v(s) = \dfrac{K e^{-Ls}}{\tau s + 1}$.

1. Ganho estático: $K = v_{enc,ss}/v_{cmd,ss}$, com `ss` = média na janela final (fração configurável, ex.: 20%).
2. Tempo morto: $L = t_{enc}^{5\%} - t_{cmd}^{5\%}$, usando primeiros cruzamentos de 5% dos valores de regime.
3. Constante de tempo: $\tau = t_{63} - L$, onde $t_{63}$ é o primeiro cruzamento de `v_enc` em 63,2% do regime.

Justificativas:
- 5% como threshold inicial reduz falso-positivos por ruído; 10% também é comum.
- 63,2% decorre da resposta de 1ª ordem: $1 - e^{-1} = 0,632$.

Heurísticas de robustez:
- Se `L ≤ 0` ou ausente, `L ← min(Δt)` (piso de resolução).
- Se `τ ≤ 0` ou ausente, `τ ← min(Δt)`.
- Sinais inválidos/NaN são descartados da média de regime.

Diagnóstico rápido:
- `v_cmd,ss` ≈ 0: degrau muito baixo → aumente setpoint.
- `v_enc` saturado: avalie `MAX_SPEED_MARGIN` no simulador/firmware.
- `L` e `τ` iguais a `min(Δt)`: telemetria muito lenta para captar a dinâmica.

## 4) Síntese PD (Ziegler–Nichols, Curva de Reação)
$K_p = 1.2 \cdot \dfrac{\tau}{K \cdot L}$,  $T_d = 0.5 L$,  $K_d = K_p T_d$,  $K_i = 0$.

Notas práticas:
- Inicie com `Kp_safe = 0.5 Kp`, aumente até `Kp` conforme estabilidade.
- Para sinais ruidosos, considere derivada filtrada (no firmware há `kd_alpha_bits`).
- Em rampas pronunciadas na fonte (degeneração de degrau), espere desvios em L/τ.

Conversão para firmware (`K_SCALE = 256`):
`kp_i = round(Kp·256)`, `ki_i = round(Ki·256)`, `kd_i = round(Kd·256)`.

Impacto do CPR do encoder em Kp:
- `K_th ≈ enc_cpr / (steps_per_rev_base · microstep)`. Menor `enc_cpr` → menor `K` → maior `Kp`.
- Ex.: Y=5000 vs X/Z=40000 implica ~8× em `Kp` para microstep e dinâmica similares.

## 5) Summary e Artefatos
- `analysis_summary.txt`: K, L, τ, ganhos PD, estatísticas de `Δt`, classificação de forma.
- `analysis_data/*_derived.csv`: séries derivadas (tempo, Δt, `v_cmd`, `v_enc`).
- Úteis para auditoria em Excel/Matplotlib e troubleshooting.

## 6) Consolidação em tuning_profiles.py
- Tabela `(axis, microstep) → (Kp, Ki, Kd, L, τ, steady_*)` consolidada manualmente.
- Consumida por `fopdt_simulator.py` e `interactive_sim.py`.
- Futuro: autocarregamento do `analysis_summary.txt` com fallback para defaults.

## Checklist de Diagnóstico
- Sanitização: há linhas suficientes? `id` cresce sempre? Um único eixo?
- Janela/stride: Δt suficientemente pequeno? ruído aceitável?
- Regime: `v_cmd,ss` e `v_enc,ss` bem definidos (sem saturação)?
- Crossings: thresholds coerentes (5% e 63,2%) atingidos?
- Ganhos: `Kp_safe` funciona no simulador? overshoot aceitável?
- Firmware: inteiros coerentes com `K_SCALE=256`?

In [ ]:
# Mini-demo: ler um *_filtered.csv, identificar K, L, τ e calcular ganhos PD
from __future__ import annotations
from pathlib import Path
import csv, math

def _load_rows(p: Path):
    out = []
    with p.open() as f:
        r = csv.reader(f)
        for c in r:
            if len(c)<5: continue
            try:
                out.append(dict(axis=int(c[0]), id=int(c[1]), time_ms=int(c[2]), encoder=int(c[3]), pulses=int(c[4])))
            except:
                pass
    return out

def _raw_series(rows):
    T=[]; Vc=[]; Ve=[]; dts=[]
    prev=None
    for r in rows:
        if prev is None: prev=r; continue
        dt_ms = r['time_ms']-prev['time_ms']
        if dt_ms<=0: prev=r; continue
        dt = dt_ms/1000.0
        Vc.append((r['pulses']-prev['pulses'])/dt)
        Ve.append((r['encoder']-prev['encoder'])/dt)
        T.append(r['time_ms']/1000.0)
        dts.append(dt)
        prev=r
    return T,Vc,Ve,dts

def _mean_tail(v, frac=0.2):
    if not v: return None
    n=max(1,int(len(v)*frac)); tail=[x for x in v[-n:] if not(math.isnan(x) or math.isinf(x))]
    if not tail: return None
    return sum(tail)/len(tail)

def _cross(T,V,thr):
    if thr is None: return None
    for t,x in zip(T,V):
        if not(math.isnan(x) or math.isinf(x)) and x>=thr: return t
    return None

def identify(T,Vc,Ve,dts):
    Vc_ss=_mean_tail(Vc); Ve_ss=_mean_tail(Ve)
    K=(Ve_ss/Vc_ss) if (Vc_ss and Vc_ss>0) else None
    tc=_cross(T,Vc,(Vc_ss*0.05) if (Vc_ss and Vc_ss>0) else None)
    te=_cross(T,Ve,(Ve_ss*0.05) if (Ve_ss and Ve_ss>0) else None)
    L=(te-tc) if (tc is not None and te is not None) else None
    if L is not None and L<0: L=0.0
    t63=_cross(T,Ve,(Ve_ss*0.632) if Ve_ss else None)
    tau=(t63-L) if (t63 is not None and L is not None) else None
    if (tau is not None and tau<0): tau=None
    mdt=min(dts) if dts else None
    if (L is None or L<=0) and mdt: L=mdt
    if (tau is None or tau<=0) and mdt: tau=mdt
    return K,L,tau,Vc_ss,Ve_ss

def zn_pd(K,L,tau):
    if not(K and L and tau) or K<=0 or L<=0 or tau<=0: return None,0.0,None
    Kp=1.2*(tau/(K*L)); Td=0.5*L; Kd=Kp*Td; return Kp,0.0,Kd

base = Path.cwd()/ 'CNC_Controller' / 'SWV_export'
cand = sorted(base.glob('*_filtered.csv')) if base.exists() else []
if not cand:
    print('Sem *_filtered.csv — rode filter_swv_export.py primeiro.')
else:
    p=cand[0]; print('Arquivo de exemplo:', p.name)
    rows=_load_rows(p); T,Vc,Ve,dts=_raw_series(rows)
    K,L,tau,_,_=identify(T,Vc,Ve,dts)
    Kp,Ki,Kd=zn_pd(K,L,tau)
    print('K=',K,' L=',L,' tau=',tau)
    print('PD Z–N: Kp=',Kp,' Ki=',Ki,' Kd=',Kd)


## FOPDT em detalhes

**Definição**
- FOPDT (First-Order Plus Dead Time) modela processos dominados por um pólo de 1ª ordem com um atraso puro (tempo morto) L.
- Intuição: a saída fica parada até L e, então, converge suavemente ao novo regime com dinâmica governada por τ.

**Modelo e equações**
- Transferência contínua: $G(s) = \dfrac{K e^{-Ls}}{\tau s + 1}$.\
  - $K$: ganho estático; $L$: atraso puro; $\tau$: constante de tempo.
- Resposta a degrau de amplitude $u_0$:\
  - $y(t) = 0$, para $t < L$\
  - $y(t) = K u_0 \big[1 - e^{-(t-L)/\tau}\big]$, para $t \ge L$

**Características no tempo**
- 63,2% (regra de $\tau$): $t_{63} = L + \tau$ (pois $1-e^{-1} = 0{,}632$).
- 90%: $t_{90} \approx L + 2{,}303\,\tau$.
- Rise time 10–90%: $t_r \approx 2{,}2\,\tau$ (desconsiderando L).
- Tempo de acomodação (±2%): $t_s \approx L + 4\,\tau$.
- Sem controlador, FOPDT puro não gera overshoot.

**Domínio da frequência**
- $G(j\omega) = \dfrac{K}{1 + j\omega\tau} \cdot e^{-j\omega L}$.
- Módulo: $|G| = \dfrac{K}{\sqrt{1 + (\omega\tau)^2}}$. Fase: $\phi(\omega) = -\arctan(\omega\tau) - \omega L$.
- O atraso reduz margem de fase (termo $-\omega L$), limitando agressividade do controle.

**Discretização (controle digital)**
- Amostragem $T_s$; atraso em passos: $n_d = \mathrm{round}(L/T_s)$.
- Pólo discreto: $\alpha = 1 - e^{-T_s/\tau}$.
- Atualização com atraso na entrada:\
  - $x[k] = x[k-1] + \alpha\, (K\,u[k-n_d] - x[k-1])$, $y[k] = x[k]$.
- Implementação no projeto: cálculo de $\alpha$ e fila de atraso em `fopdt_simulator.py`.

**Identificação experimental (passo a passo)**
- Aplique degrau na entrada e meça saída. No projeto, derivamos velocidades de `pulses` e `encoder`.
- Ganho $K = v_{enc,ss}/v_{cmd,ss}$ (médias em regime).
- Atraso $L = t_{enc}^{5\%} - t_{cmd}^{5\%}$ (primeiros cruzamentos em 5% do regime).
- Constante $\tau = t_{63} - L$, onde $t_{63}$ é o cruzamento a 63,2% da saída.
- Robustez: se $L \le 0$ ou $\tau \le 0$, use piso $\min(\Delta t)$ (resolução dos dados).

**Relação com PID/tuning**
- Ziegler–Nichols (curva de reação) para PD (Ki=0): $K_p = 1{,}2 \cdot \dfrac{\tau}{K L}$, $T_d = 0{,}5 L$, $K_d = K_p T_d$.
- Alternativas (IMC, Cohen–Coon) tendem a ser mais robustas quando $L/\tau$ é grande.
- Conversão firmware (escala 256): $kp_i=\mathrm{round}(K_p\cdot256)$ etc.

**Quando usar / limitações**
- Adequado para processos com pólo dominante + atraso (térmico, fluxo, servo de velocidade).
- Menos adequado para integradores puros (use IDT) ou 2ª ordem subamortecida com oscilações fortes.
- Atrasos grandes ($L/\tau$ alto) estreitam margens; use ganhos conservadores.

**Exemplos práticos**
- Aquecedor (inércia térmica e atraso de transporte), linhas de fluxo, servo com latências de amostragem/atuadores.

**Conexão com este projeto**
- O plant de velocidade é tratado como FOPDT; identificamos $K, L, \tau$ de SWV.
- O CPR do encoder reduz $K$ quando menor (ex.: 5000 vs 40000) → exige $K_p$ maior, mantidos $L$ e $\tau$.
- Implementações: FOPDT discreto em `fopdt_simulator.py`; identificação + Z–N em `analyze_step_responses.py`; consolidação em `tuning_profiles.py`.


## FAQ
- Por que 5% e 63,2%? São thresholds canônicos: 5–10% para início de resposta e 63,2% para τ em sistemas de 1ª ordem.
- Dá para usar Ki ≠ 0? Sim, mas aqui o integrador está na malha de posição; somar Ki na malha de velocidade pode causar interação indesejada.
- Kp do Y é maior por quê? CPR menor → K menor → Kp maior (vide seção 4).
- E se tiver muito overshoot? Rebaixe Kp (ex.: 0.5×) e/ou aumente derivada (Kd) considerando filtro de derivada no firmware.